# 영상 필터링 (잡음 제거)

잡음 제거는 영상처리에서 전처리 필수 요소입니다. 잡음 제거를 위해 사용할 수 있는 효과적인 방법 몇 가지를 정리해보았습니다. 

## 평균값 필터

주목 화소의 주위 8근방 화소의 평균(Mean)을 이용해서 잡음을 제거합니다. 

## 블러 필터 

### cv2.blur(src, ksize, borderType) -> dst

커널 영역의 모든 픽셀의 평균을 계산하고 현재 주목하는 중앙 화소를 이 평균으로 대체합니다. 이 함수는 커널의 너비와 높이를 지정해야 합니다. 

- src : 입력 이미지입니다.
- ksize : 필터 커널의 크기입니다.
- borderType : 이미지 외부의 픽셀을 추정하는데 사용되는 테두리 모드입니다. cv2.BORDER_XXX 형식입니다.

### cv2.medianBlur(src, ksize, borderType) -> dst 

메디안 필터는 주목 화소와 8근방 화소의 중앙값(Median)을 이용해서 잡음을 제거합니다. 

In [ ]:
# 평균값 필터
import numpy as np
import cv2

def mean_blur(img):
    A = np.zeros(img.shape, dtype=np.uint8)
    height, width = img.shape
    for y in range(height):
        for x in range(width):
            try :
                S = 1*img[y-1, x-1] + img[y-1, x] + img[y-1, x+1] \
                    + img[y  , x-1] + img[y  , x] + img[y  , x+1] \
                    + img[y+1, x-1] + img[y+1, x] + img[y+1, x+1]
                S = S/9
                if S>255:
                    A[y,x] = 255
                elif S<0:
                    A[y,x] = 0
                else:
                    A[y,x] = int(S)
            except:
                pass
    return A

lena_gray = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Lena", lena_gray)
cv2.imshow("Mean Blur", mean_blur(lena_gray))
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 블러 필터
import numpy as np
import cv2

lena_gray = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Lena", lena_gray)
cv2.imshow("Mean Blur", cv2.blur(lena_gray, (3,3)))
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
# 메디안 필터
import numpy as np
import cv2

lena_gray = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)
result = cv2.medianBlur(lena_gray, 3)
cv2.imshow("Lena", lena_gray)
cv2.imshow("Median filter", result)
cv2.waitKey()
cv2.destroyAllWindows()

## 가우시안 필터

가우시안 분포를 갖는 커널을 이용해서 필터 처리를 할 수 있습니다. 가우시안 필터는 중앙값이 가장 크고 멀어질수록 그 값이 작아지는 커널을 사용합니다. 현재 주목 화소에 가중치를 더 부여하고 현재 화소와 상하좌우 화소의 길이가 1이라면 대각선 성분의 길이는 2의 제곱근이므로 상하좌우 화소는 대각선 화소에 비해 더 높은 가중치를 줍니다. 

### cv2. GaussianBlur(src, ksize, sigmaX, sigmaY, borderType) -> dst

- src : 8비트 1채널 또는 3채널 입력 이미지입니다.
- ksize : 필터 커널의 크기입니다.
- sigmaX, sigmaY : X 방향과 Y 방향 가우시안 커널 표준편차입니다.
- borderType : 이미지 외부의 픽셀을 추정하는데 사용되는 테두리 모드입니다. cv2.BORDER_XXX 형식입니다.

## 바이래터럴 필터

cv2.bilateralFilter() 함수는 가장자리를 선명하게 유지하면서 노이즈 제거에 매우 효과적입니다. 그러나 작업 속도가 다른 필터에 비해 느립니다(느껴질 정도입니다). 

가우시안 필터는 공간 만을 고려한 함수입니다. 즉, 필터링하는 동안 주변 픽셀이 고려됩니다. 픽셀의 강도가 거의 동일한지 여부는 고려하지 않습니다. 바이래터럴(양방향) 필터링은 공간에서 가우시안 필터를 사용하지만 화소 차이의 함수인 가우시안 필터를 하나 더 사용합니다. 공간의 가우시안 함수는 주변 화소만 블러링에 고려하는 반면, 강도 차이의 가우시안 함수는 중앙 화소와 유사한 강도를 가진 화소만 블러링에 고려합니다.

### cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace, borderType) -> dst

- src : 8비트 1채널 또는 3채널 입력 이미지입니다.
- d : 필터링 시 고려할 주변 화소의 지름입니다.
- sigmaColor : 색 공간에서 시그마를 필터링합니다. 매개 변수의 값이 클수록 픽셀 주변 내에서 더 멀리 떨어진 색상이 함께 혼합되어 반균등 색상의 영역이 더 크게 형성됩니다. 
- sigmaSpace : 좌표 공간에서 시그마를 필터링합니다. 매개 변수의 값이 클수록 더 먼 픽셀이 영향을 끼칩니다.
- borderType : 이미지 외부의 픽셀을 추정하는데 사용되는 테두리 모드입니다. cv2.BORDER_XXX 형식입니다.

In [ ]:
# 가우시안 필터 직접 구현
import numpy as np
import cv2

def gaussian_blur(img):
    img_ = np.zeros(img.shape, dtype=np.uint8)
    height, width = img.shape
    for y in range(height):
        for x in range(width):
            try :
                T =  1*img[y-1, x-1] +  2*img[y-1, x] +  1*img[y-1, x+1] \
                  +  2*img[y  , x-1] +  4*img[y  , x] +  2*img[y  , x+1] \
                  +  1*img[y+1, x-1] +  2*img[y+1, x] +  1*img[y+1, x+1] 
                img_[y,x] = int(T/16)
            except:
                pass
    return img_

lena_gray = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)

cv2.imshow("Lena", lena_gray)
cv2.imshow("Gaussian Blur", gaussian_blur(lena_gray))
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
import cv2

lena_gray = cv2.imread("./data/lena.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Gaussian", cv2.GaussianBlur(lena_gray, (5, 5), 75))
cv2.imshow("Bilateral", cv2.bilateralFilter(lena_gray, 9, 75, 75))
cv2.waitKey()
cv2.destroyAllWindows()

# 모폴로지 연산

모폴로지(Morphology) 연산은 형태학적 변환을 의미합니다. 수축과 팽창이 이미지에 어떤 영향을 주는지 확인할 수 있습니다. 

## 수축 

수축은 화소의 부근에 하나라도 0인 화소가 있으면 그 화소를 0으로, 그 외에는 255로 처리합니다. 수축 후 물체의 두께 또는 크기가 감소하거나 흰색 영역이 이미지에서 감소합니다. 수축은 작은 백색 잡음을 제거하고 두 개의 연결된 개체를 분리하는 데 유용하게 쓰입니다.

## 팽창

팽창은 수축의 반대입니다(당연). 화소의 부근에 하나라도 255가 있으면 그 화소를 255로, 그 외는 0으로 처리합니다. 따라서 이미지의 흰색 영역이 증가하거나 전경 물체의 크기가 증가합니다. 

In [13]:
# 수축 함수 구현 및 적용
def contraction(img_in):
    height, width = img_in.shape
    img_out = np.zeros(img_in.shape, np.uint8)
    for y in range(1, height-1):
        for x in range(1, width-1):
            img_out[y,x] = img_in[y,x]
            if img_in[y-1,x-1]==0:
                img_out[y,x] = 0
            if img_in[y-1,x  ]==0:
                img_out[y,x] = 0
            if img_in[y-1,x+1]==0:
                img_out[y,x] = 0
                
            if img_in[y  ,x-1]==0:
                img_out[y,x] = 0
            if img_in[y  ,x+1]==0:
                img_out[y,x] = 0
                
            if img_in[y+1,x-1]==0:
                img_out[y,x] = 0
            if img_in[y+1,x  ]==0:
                img_out[y,x] = 0
            if img_in[y+1,x+1]==0:
                img_out[y,x] = 0
    return img_out

imgMPG = cv2.imread("./data/morphology.jpg", cv2.IMREAD_GRAYSCALE)
imgMPG_cont = contraction(imgMPG)
cv2.imshow("MPG", imgMPG)
cv2.imshow("imgMPG_cont", imgMPG_cont)
cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
# 팽창 함수 구현 및 적용
def expansion(img_in):
    height, width = img_in.shape
    img_out = np.zeros(img_in.shape, np.uint8)
    for y in range(1, height-1):
        for x in range(1, width-1):
            img_out[y,x] = img_in[y,x]
            if img_in[y-1,x-1]==255:
                img_out[y,x] = 255
            if img_in[y-1,x  ]==255:
                img_out[y,x] = 255
            if img_in[y-1,x+1]==255:
                img_out[y,x] = 255
                
            if img_in[y  ,x-1]==255:
                img_out[y,x] = 255
            if img_in[y  ,x+1]==255:
                img_out[y,x] = 255
                
            if img_in[y+1,x-1]==255:
                img_out[y,x] = 255
            if img_in[y+1,x  ]==255:
                img_out[y,x] = 255
            if img_in[y+1,x+1]==255:
                img_out[y,x] = 255
    return img_out

imgMPG = cv2.imread("./data/morphology.jpg", cv2.IMREAD_GRAYSCALE)
imgMPG_cont = expansion(imgMPG)
cv2.imshow("MPG", imgMPG)
cv2.imshow("imgMPG_cont", imgMPG_cont)
cv2.waitKey()
cv2.destroyAllWindows()

## 수축과 팽창을 위한 OpenCV 함수

### cv2.erode(src, dst, kernel, anchor=(-1, -1), iteration=1)

수축을 위한 함수입니다. 

- src : 입력 이미지입니다. 채널 수는 임의로 지정할 수 있지만 깊이는 CV_XXX 형식을 사용해야 합니다. 
- dst : src와 동일한 크기 및 타입의 출력 이미지입니다. 
- kernel : 침식에 사용되는 필터커널입니다. 
- anchor : 요소 내 현재 화소의 위치입니다. 기본값 (-1, -1)은 현재 화소가 필터 커널의 중심에 있음을 의미합니다.
- iteration : 침식이 적용되는 횟수입니다.

### cv2.dilate(src, dst, kernel, anchor=(-1, -1), iteration=1)

팽창을 위한 함수입니다. 

- src : 입력 이미지입니다. 채널 수는 임의로 지정할 수 있지만 깊이는 CV_XXX 형식을 사용해야 합니다. 
- dst : src와 동일한 크기 및 타입의 출력 이미지입니다. 
- kernel : 침식에 사용되는 필터커널입니다. 
- anchor : 요소 내 현재 화소의 위치입니다. 기본값 (-1, -1)은 현재 화소가 필터 커널의 중심에 있음을 의미합니다.
- iteration : 침식이 적용되는 횟수입니다.

In [21]:
import cv2
import numpy as np
img = cv2.imread("./data/morphology.jpg", cv2.IMREAD_GRAYSCALE)
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)
dilation = cv2.dilate(img,kernel,iterations = 1)

cv2.imshow("erosion", erosion)
cv2.imshow("dilation", dilation)
cv2.waitKey()
cv2.destroyAllWindows()